zad1.py

In [ ]:
import cv2
from matplotlib import pyplot as plt
from functions import calcPSNR

masks = [(3, 3), (5, 5), (7, 7)]

img_original = cv2.imread('./images/girl_col.png', cv2.IMREAD_UNCHANGED)
img_noise = cv2.imread('./images/girl_col_noise.png', cv2.IMREAD_UNCHANGED)
img_inoise = cv2.imread('./images/girl_col_inoise.png', cv2.IMREAD_UNCHANGED)

psnr_noise = calcPSNR(img_original, img_noise)
psnr_inoise = calcPSNR(img_original, img_inoise)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
axes[0].set_title("Oryginał")
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_noise, cv2.COLOR_BGR2RGB))
axes[1].set_title(f"Szum Gaussowski (PSNR: {psnr_noise:.3f})")
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_inoise, cv2.COLOR_BGR2RGB))
axes[2].set_title(f"Szum Impulsowy (PSNR: {psnr_inoise:.3f})")
axes[2].axis('off')

plt.tight_layout()
plt.savefig("./images/zad1_oryginaly.png")
plt.close()

for mask in masks:
    img_gaussian_blur = cv2.GaussianBlur(img_noise, mask, 0)
    psnr_gauss = calcPSNR(img_original, img_gaussian_blur)

    img_median_blur = cv2.medianBlur(img_noise, mask[0])
    psnr_median = calcPSNR(img_original, img_median_blur)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    axes[0].imshow(cv2.cvtColor(img_gaussian_blur, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f"Filtr Gaussa {mask} (PSNR: {psnr_gauss:.2f})")
    axes[0].axis('off')

    axes[1].imshow(cv2.cvtColor(img_median_blur, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f"Filtr medianowy {mask} (PSNR: {psnr_median:.2f})")
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(f"./images/zad1_noise_maska_{mask[0]}x{mask[1]}.png")
    plt.close()

    img_gaussian_blur_inoise = cv2.GaussianBlur(img_inoise, mask, 0)
    psnr_gauss_inoise = calcPSNR(img_original, img_gaussian_blur_inoise)

    img_median_blur_inoise = cv2.medianBlur(img_inoise, mask[0])
    psnr_median_inoise = calcPSNR(img_original, img_median_blur_inoise)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    axes[0].imshow(cv2.cvtColor(img_gaussian_blur_inoise, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f"Filtr Gaussa {mask} (PSNR: {psnr_gauss_inoise:.2f})")
    axes[0].axis('off')

    axes[1].imshow(cv2.cvtColor(img_median_blur_inoise, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f"Filtr medianowy {mask} (PSNR: {psnr_median_inoise:.2f})")
    axes[1].axis('off')

    plt.tight_layout()
    plt.savefig(f"./images/zad1_inoise_maska_{mask[0]}x{mask[1]}.png")
    plt.close()


zad2.py

In [ ]:
import cv2
from matplotlib import pyplot as plt
from functions import calcPSNR


img_original = cv2.imread('./images/girl_col.png', cv2.IMREAD_UNCHANGED)

img_yuv = cv2.cvtColor(img_original, cv2.COLOR_BGR2YUV)

img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])

img_equalized = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

cv2.imwrite('./images/zad2_girl_col_equalized.png', img_equalized)

psnr_value = calcPSNR(img_original, img_equalized)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title("Obraz oryginalny")
axes[0, 0].axis('off')

axes[0, 1].imshow(cv2.cvtColor(img_equalized, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title("Obraz po wyrównaniu histogramu")
axes[0, 1].axis('off')

hist_original = cv2.calcHist([img_original], [0], None, [256], [0, 256])
hist_original = hist_original.flatten()
hist_equalized = cv2.calcHist([img_equalized], [0], None, [256], [0, 256])
hist_equalized = hist_equalized.flatten()

axes[1, 0].plot(hist_original)
axes[1, 0].set_title("Histogram - obraz oryginalny")
axes[1, 0].set_xlim([0, 256])
axes[1, 0].set_ylim([0, 9000])
axes[1, 0].grid(True)

axes[1, 1].plot(hist_equalized)
axes[1, 1].set_title("Histogram - po wyrównaniu")
axes[1, 1].set_xlim([0, 256])
axes[1, 1].set_ylim([0, 9000])
axes[1, 1].grid(True)


plt.tight_layout()
plt.savefig("./images/zad2_histogramy.png")
plt.close()


zad3.py

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_original = cv2.imread("./images/girl_col.png")

img_original_rgb = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

weights = [-0.5, -1, -1.5, -2, -3]

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

axes[0].imshow(img_original_rgb)
axes[0].set_title("Obraz oryginalny")
axes[0].axis('off')

for i, weight in enumerate(weights):
    img_laplace = cv2.Laplacian(img_original, cv2.CV_64F)
    image = np.asarray(img_original, np.float64)
    img_out = cv2.addWeighted(image, 1, img_laplace, weight, 0)
    img_out = np.clip(img_out, 0, 255).astype(np.uint8)

    img_out_rgb = cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB)
    axes[i+1].imshow(img_out_rgb)
    axes[i+1].set_title(f"Waga = {weight}")
    axes[i+1].axis('off')

plt.tight_layout()
plt.savefig("./images/zad3_wyostrzone.png")
plt.show()

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

axes[0].imshow(img_original_rgb)
axes[0].set_title("Obraz oryginalny")
axes[0].axis('off')

for i, weight in enumerate(weights):
    img_gauss = cv2.GaussianBlur(img_original, (3, 3), 0)
    img_laplace = cv2.Laplacian(img_gauss, cv2.CV_64F)
    image = np.asarray(img_original, np.float64)
    img_out = cv2.addWeighted(image, 1, img_laplace, weight, 0)
    img_out = np.clip(img_out, 0, 255).astype(np.uint8)

    img_out_rgb = cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB)
    axes[i+1].imshow(img_out_rgb)
    axes[i+1].set_title(f"Waga = {weight} (wygładzone)")
    axes[i+1].axis('off')

plt.tight_layout()
plt.savefig("./images/zad3_wyostrzone_gauss.png")
plt.show()

best_weight = -3

img_laplace = cv2.Laplacian(img_original, cv2.CV_64F)
img_gauss = cv2.GaussianBlur(img_original, (3, 3), 0)
img_laplace_smooth = cv2.Laplacian(img_gauss, cv2.CV_64F)

image = np.asarray(img_original, np.float64)
img_out_no_smooth = cv2.addWeighted(image, 1, img_laplace, best_weight, 0)
img_out_no_smooth = np.clip(img_out_no_smooth, 0, 255).astype(np.uint8)
img_out_smooth = cv2.addWeighted(image, 1, img_laplace_smooth, best_weight, 0)
img_out_smooth = np.clip(img_out_smooth, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(img_original_rgb)
axes[0].set_title("Obraz oryginalny")
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(img_out_no_smooth, cv2.COLOR_BGR2RGB))
axes[1].set_title(f"Waga = {best_weight}")
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_out_smooth, cv2.COLOR_BGR2RGB))
axes[2].set_title(f"Waga = {best_weight} (wygładzone)")
axes[2].axis('off')

plt.tight_layout()
plt.savefig("./images/zad3_porownanie.png")
plt.show()


functions.py

In [ ]:
import numpy as np


def calcPSNR(img1, img2):
    imax = 255.**2
    mse = ((img1.astype(np.float64)-img2)**2).sum()/img1.size
    return 10.0*np.log10(imax/mse)
